In [18]:
# Importation des bibliothèques nécessaires
import json
import re
import unicodedata
import pandas as pd
import nltk 
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('french'))

In [26]:
df = pd.read_json('../data/raw/tweets_fr.json',lines=True)
df

,tweet_id,text,created_at,author_id,context_annotations,language
0,1858189684943265825,@wargonm Un peu toi pendant le Covid non ? Il ...,2024-11-17 16:45:19+00:00,1801562435813281793,"[{'domain': {'id': '123', 'name': 'Ongoing New...",français
1,1858189572150067631,Il a raison les gens ne voient que ceux que le...,2024-11-17 16:44:52+00:00,1283275292019228672,"[{'domain': {'id': '10', 'name': 'Person', 'de...",français
2,1858189551329493279,Un lanceur d'alerte a obtenu 10 Go de l'Instit...,2024-11-17 16:44:47+00:00,1840001168438689792,"[{'domain': {'id': '123', 'name': 'Ongoing New...",français
3,1858189437819347092,"COVID, Effets Secondaires, Vacc*ns : Les derni...",2024-11-17 16:44:20+00:00,3394102223,"[{'domain': {'id': '123', 'name': 'Ongoing New...",français
4,1858189382278156770,C'est la technique imparable\nde la caste au P...,2024-11-17 16:44:06+00:00,206513780,"[{'domain': {'id': '123', 'name': 'Ongoing New...",français
5,1858189369854595534,@lonnibesancon @Clarivate C'est toi la fraudul...,2024-11-17 16:44:03+00:00,1428599123348004864,None,français
6,1858189363223683086,"@CNEWS Non seulement il parle, il a grossit......",2024-11-17 16:44:02+00:00,1400182391487045636,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",français
7,1858189359700201912,@raoult_didier vous n'êtes pas credible vous a...,2024-11-17 16:44:01+00:00,548728549,"[{'domain': {'id': '123', 'name': 'Ongoing New...",français
8,1858189250023354659,@VieDeCarabin J'ai simplement critiqué ton des...,2024-11-17 16:43:35+00:00,1678100076227379201,"[{'domain': {'id': '123', 'name': 'Ongoing New...",français
9,1858189193085686010,@LeFigaro_Sante Les effets secondaires n'exist...,2024-11-17 16:43:21+00:00,1698633392118439936,"[{'domain': {'id': '123', 'name': 'Ongoing New...",français


In [3]:
# Normalisation du texte
def normalize_text(text, language):
    """
    Nettoie et normalise un texte en fonction de la langue.
    - Supprime les caractères indésirables et unifie les styles d'écriture.
    """
    if language == "arabe":
        text = unicodedata.normalize("NFKD", text)
        text = re.sub(r"[إأآا]", "ا", text)
        text = re.sub(r"ؤ", "و", text)
        text = re.sub(r"ئ", "ي", text)
    elif language in ["francais", "anglais"]:
        text = text.lower()  # Met en minuscule
    text = re.sub(r"http\S+", "", text)  # Supprime les URL
    text = re.sub(r"@\w+", "", text)  # Supprime les mentions
    text = re.sub(r"#\w+", "", text)  # Supprime les hashtags
    text = re.sub(r"[^a-zA-Z\u0621-\u064A\s]", "", text)  # Supprime les caractères non-alphabétiques
    return text.strip()

In [14]:
# Nettoyage des tweets
def clean_tweets(filepath, language, stopwords):
    """
    Nettoie les tweets d'un fichier JSON.
    - filepath : chemin du fichier JSON
    - language : langue des tweets (arabe, francais, anglais)
    - stopwords : liste des mots vides
    """
    cleaned_data = []

    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            try:
                # Essayer de charger comme un tableau JSON
                data = json.load(file)
            except json.JSONDecodeError:
                # Si échec, charger chaque ligne comme un objet JSON
                file.seek(0)  # Réinitialiser le curseur
                data = [json.loads(line) for line in file]

        for tweet in data:
            # Valider que le tweet a un champ "text" et un champ "id"
            if not isinstance(tweet, dict) or "text" not in tweet or "id" not in tweet:
                print(f"Tweet malformé ignoré : {tweet}")
                continue

            text = tweet["text"]
            tweet_id = tweet["id"]

            # Normaliser le texte en fonction de la langue
            clean_text = normalize_text(text, language)
            # Supprimer les mots vides
            clean_text = " ".join([word for word in clean_text.split() if word not in stopwords])

            # Ajouter le tweet nettoyé au résultat
            cleaned_data.append({"id": tweet_id, "text": clean_text})

    except FileNotFoundError:
        print(f"Erreur : Le fichier {filepath} est introuvable.")
    except Exception as e:
        print(f"Erreur inattendue lors du nettoyage : {e}")

    return cleaned_data

